In [3]:
import chess
import chess.pgn
import pandas as pd
import itertools

In [4]:
# Set up square data.
files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
ranks = ['1', '2', '3', '4', '5', '6', '7', '8']
squares = [''.join(s) for s in itertools.product(*[files,ranks])]
print(squares)

['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8']


In [56]:
# Set up occupants keys. 
# All of the pieces of each color, plus an empty disignation.
occupants = ['wr', 'wn', 'wb', 'wq', 'wk', 'wp', 'br', 'bn', 'bb', 'bq', 'bk', 'bp', 'x']

In [6]:
# Location of downloaded pgn database file.
database_filename = 'C:/Users/alexpc2red/Documents/lichessgames/lichess_db_standard_rated_2016-03.pgn'

In [37]:
# Define a conversion from library color notation to ours.
def parse_piece_color(piece):
    if piece.color is None:
        return None
    elif piece.color:
        return 'w'
    else:
        return 'b'

In [115]:
# Determine if board state has not been seen yet.
# Return -1 if new board state, otherwise return index of seen board
def index_of_board_state(board_states, board_state_new):
    try: 
        return board_states.index(board_state_new)
    except ValueError:
        return -1

In [72]:
# Create new board state object representing empty board.
def generate_empty_board_state():
    board_state = {}
    for square in squares:
        board_state[square] = 'x'
        board_state[square + '_occupant'] = 'x'
    return board_state

In [59]:
def parse_board_state(board):
    # Open database file.
    board_state =  generate_empty_board_state()
    for square in squares:
        board_state[square] = 'x'
        board_state[square + '_occupant'] = 'x'
        piece = board.piece_at(chess.parse_square(square))
        if piece is not None:
            color = parse_piece_color(piece)
            piece_char = piece.symbol().lower()
            occupant = color + piece_char
            board_state[square] = color
            board_state[square + '_occupant'] = occupant
    return board_state

In [119]:
def parse_all_unique_board_states(max_games=-1):
    board_states = []
    counts = []
    total_games = 0
    pgn = open(database_filename)
    while True:
        if total_games > max_games and max_games > 0:
            break
        game = chess.pgn.read_game(pgn)
        print(len(list(game.mainline_moves())))
        if game is None:
            break
        board = game.board()
        board_state = parse_board_state(board)
        idx = index_of_board_state(board_states,board)
        if idx == -1:
            board_states.append(board_state)
            counts.append(1)
        else:
            counts[idx] += 1
        if board_state not in board_states:
            board_states.append(board_state)
        for move in game.mainline_moves():
            board.push(move)
            board_state = parse_board_state(board)
            idx = index_of_board_state(board_states,board)
            if idx == -1:
                board_states.append(board_state)
                counts.append(1)
            else:
                counts[idx] += 1
            if board_state not in board_states:
                board_states.append(board_state)
        total_games += 1 
    return board_states,counts

In [120]:
board_states,counts = parse_all_unique_board_states(5)
print(f'total board_states: {len(board_states)}')
print(f'total counts: {len(counts)}')

0
77
55
54
92
119
total board_states: 403
total counts: 403


In [121]:
board_states[0] == board_states[1]

True

403
403
1


In [101]:
board_states[0] == board_states[2]

False

In [87]:
print(len(board_states))

403


In [92]:
len(counts)

403

In [ ]:
board_states[1]

In [89]:
print(board_states[5])

{'a1': 'w', 'a1_occupant': 'wr', 'a2': 'w', 'a2_occupant': 'wp', 'a3': 'x', 'a3_occupant': 'x', 'a4': 'x', 'a4_occupant': 'x', 'a5': 'x', 'a5_occupant': 'x', 'a6': 'x', 'a6_occupant': 'x', 'a7': 'b', 'a7_occupant': 'bp', 'a8': 'b', 'a8_occupant': 'br', 'b1': 'w', 'b1_occupant': 'wn', 'b2': 'w', 'b2_occupant': 'wp', 'b3': 'x', 'b3_occupant': 'x', 'b4': 'x', 'b4_occupant': 'x', 'b5': 'x', 'b5_occupant': 'x', 'b6': 'x', 'b6_occupant': 'x', 'b7': 'b', 'b7_occupant': 'bp', 'b8': 'b', 'b8_occupant': 'bn', 'c1': 'w', 'c1_occupant': 'wb', 'c2': 'w', 'c2_occupant': 'wp', 'c3': 'x', 'c3_occupant': 'x', 'c4': 'x', 'c4_occupant': 'x', 'c5': 'x', 'c5_occupant': 'x', 'c6': 'x', 'c6_occupant': 'x', 'c7': 'b', 'c7_occupant': 'bp', 'c8': 'b', 'c8_occupant': 'bb', 'd1': 'w', 'd1_occupant': 'wq', 'd2': 'x', 'd2_occupant': 'x', 'd3': 'w', 'd3_occupant': 'wp', 'd4': 'x', 'd4_occupant': 'x', 'd5': 'b', 'd5_occupant': 'bp', 'd6': 'x', 'd6_occupant': 'x', 'd7': 'x', 'd7_occupant': 'x', 'd8': 'b', 'd8_occupant

In [82]:
bs = parse_board_state(board)
print(bs)
print(sys.getsizeof(bs))

{'a1': 'w', 'a1_occupant': 'wr', 'a2': 'w', 'a2_occupant': 'wp', 'a3': 'x', 'a3_occupant': 'x', 'a4': 'x', 'a4_occupant': 'x', 'a5': 'x', 'a5_occupant': 'x', 'a6': 'x', 'a6_occupant': 'x', 'a7': 'b', 'a7_occupant': 'bp', 'a8': 'b', 'a8_occupant': 'br', 'b1': 'w', 'b1_occupant': 'wn', 'b2': 'w', 'b2_occupant': 'wp', 'b3': 'x', 'b3_occupant': 'x', 'b4': 'x', 'b4_occupant': 'x', 'b5': 'x', 'b5_occupant': 'x', 'b6': 'x', 'b6_occupant': 'x', 'b7': 'b', 'b7_occupant': 'bp', 'b8': 'b', 'b8_occupant': 'bn', 'c1': 'w', 'c1_occupant': 'wb', 'c2': 'w', 'c2_occupant': 'wp', 'c3': 'x', 'c3_occupant': 'x', 'c4': 'x', 'c4_occupant': 'x', 'c5': 'x', 'c5_occupant': 'x', 'c6': 'x', 'c6_occupant': 'x', 'c7': 'b', 'c7_occupant': 'bp', 'c8': 'b', 'c8_occupant': 'bb', 'd1': 'w', 'd1_occupant': 'wq', 'd2': 'w', 'd2_occupant': 'wp', 'd3': 'x', 'd3_occupant': 'x', 'd4': 'x', 'd4_occupant': 'x', 'd5': 'x', 'd5_occupant': 'x', 'd6': 'x', 'd6_occupant': 'x', 'd7': 'b', 'd7_occupant': 'bp', 'd8': 'b', 'd8_occupant

In [47]:
import sys

In [51]:
new_bs = {}
for key in bs.keys():
    new_bs[key] = 0


4696

In [52]:
import copy

In [53]:
sys.getsizeof([copy.deepcopy(bs),copy.deepcopy(bs)])

72

In [71]:
print(game1.board())

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [81]:
['a','b','c','d','e','f'][[1,2,3]]

<>:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
C:\Users\alexpc2red\AppData\Local\Temp\ipykernel_10724\3330116584.py:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
  ['a','b','c','d','e','f'][[1,2,3]]
C:\Users\alexpc2red\AppData\Local\Temp\ipykernel_10724\3330116584.py:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
  ['a','b','c','d','e','f'][[1,2,3]]
C:\Users\alexpc2red\AppData\Local\Temp\ipykernel_10724\3330116584.py:1: SyntaxWarning: list indices must be integers or slices, not list; perhaps you missed a comma?
  ['a','b','c','d','e','f'][[1,2,3]]


TypeError: list indices must be integers or slices, not list

In [116]:
ValueError

ValueError

In [122]:
board.fen()

'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'

In [123]:
board.fen().split(' ')[0]

'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'